# Caso de Negocio

### [BBT3-9](https://alkemy-labs.atlassian.net/browse/BBT3-9) Fabricio

### [BBT3-10](https://alkemy-labs.atlassian.net/browse/BBT3-10) Judith

### [BBT3-11](https://alkemy-labs.atlassian.net/browse/BBT3-11) Pablo

### Creacion notebook e importación de librerías.

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=094e5c5ddcf7dc5df7410de544e19b0ec889fbddc56bef5486c67b4ff73ae142
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [2]:
import pandas as pd
from pyspark.sql import SparkSession
import sqlite3 as sql

## Datasets a base de datos

### [BBT3-12](https://alkemy-labs.atlassian.net/browse/BBT3-12) Fabricio

### [BBT3-13](https://alkemy-labs.atlassian.net/browse/BBT3-13) Judith

### [BBT3-14](https://alkemy-labs.atlassian.net/browse/BBT3-14) Pablo

In [3]:
order_items_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_order_items_dataset.csv'
customers_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_customers_dataset.csv'
geolocation_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_geolocation_dataset.csv'
sellers_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_sellers_dataset.csv'
order_payments_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_order_payments_dataset.csv'
product_category_name_translation_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20product_category_name_translation.csv'
order_reviews_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_order_reviews_dataset.csv'
orders_link_csv = 'https://github.com/alkemyTech/BBVA-BDS-W1-T3/raw/main/datasets/Copia%20de%20ecommerce_orders_dataset.csv'
products_link_csv = 'https://raw.githubusercontent.com/alkemyTech/BBVA-BDS-W1-T3/main/datasets/Copia%20de%20ecommerce_products_dataset.csv'


order_items_df = pd.read_csv(order_items_link_csv)
order_payments_df = pd.read_csv(order_payments_link_csv)
products_df = pd.read_csv(products_link_csv)
customers_df = pd.read_csv(customers_link_csv)
orders_df = pd.read_csv(orders_link_csv)
order_reviews_df = pd.read_csv(order_reviews_link_csv)
geolocation_df = pd.read_csv(geolocation_link_csv)
sellers_df = pd.read_csv(sellers_link_csv)
product_category_name_translation_df = pd.read_csv(product_category_name_translation_link_csv)

In [4]:
# seteo de columnas indices:
order_items_df.set_index(['order_id', 'order_item_id'], inplace=True)
order_payments_df.set_index(['order_id','payment_sequential'], inplace=True)
products_df.set_index('product_id', inplace=True)
customers_df.set_index('customer_id', inplace=True)
orders_df.set_index('order_id', inplace=True)
order_reviews_df.set_index('review_id', inplace=True)
geolocation_df.set_index('geolocation_zip_code_prefix', inplace=True)
sellers_df.set_index('seller_id', inplace=True)
product_category_name_translation_df.set_index('product_category_name', inplace=True)

In [5]:
# creacion base de datos
conn = sql.connect('ecommerce.db')
cursor = conn.cursor()

In [6]:
# guardado de dataframes en tablas de la base de datos
order_payments_df.to_sql('order_payments', con=conn, if_exists='replace')
order_items_df.to_sql('order_items', con=conn, if_exists='replace')
orders_df.to_sql('orders', con=conn, if_exists='replace')
customers_df.to_sql('customers', con=conn, if_exists='replace')
products_df.to_sql('products', con=conn, if_exists='replace')
order_reviews_df.to_sql('order_reviews', con=conn, if_exists='replace')
geolocation_df.to_sql('geolocation', con=conn, if_exists='replace')
sellers_df.to_sql('sellers', con=conn, if_exists='replace')
product_category_name_translation_df.to_sql('product_category_name_translation', con=conn, if_exists='replace')

71

In [7]:
# query para obtener los nombres de las tablas creadas
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
lista_tablas = cursor.fetchall()
lista_nombre_tablas = [tabla[0] for tabla in lista_tablas]
print(lista_nombre_tablas)

['order_payments', 'order_items', 'orders', 'customers', 'products', 'order_reviews', 'geolocation', 'sellers', 'product_category_name_translation']


## Extraccion de Base de Datos


### orders_items y sellers
###[BBT3-17](https://alkemy-labs.atlassian.net/browse/BBT3-17) Judith

In [8]:
query_items_seller = '''
  SELECT * FROM order_items
  INNER JOIN sellers
  ON order_items.seller_id = sellers.seller_id
'''
items_seller_df = pd.read_sql_query(query_items_seller, conn)
items_seller_df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,48436dade18ac8b2bce089ec2a041202,27277,volta redonda,SP
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,dd7ddc04e1b6c2c614352b383efe2d36,3471,sao paulo,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,5b51032eddd242adc84c38acab88f23d,37564,borda da mata,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,9d7a1d34a5052409006425275ba1c2b4,14403,franca,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,df560393f3a51e74553ab94004ba5c87,87900,loanda,PR
...,...,...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,b8bc237ba3788b23da09c0f1f3a3288c,88303,itajai,SC
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,f3c38ab652836d21de61fb8314b69182,1206,sao paulo,SP
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,c3cfdc648177fdbbbb35635a37472c53,80610,curitiba,PR
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,2b3e4a2a3ea8e01938cabda2a3e5cc79,4733,sao paulo,SP


### orders_items y products
### [BBT3-18](https://alkemy-labs.atlassian.net/browse/BBT3-18) Judith

In [9]:
query_items_products = '''
  SELECT * FROM order_items
  INNER JOIN products
  ON order_items.product_id = products.product_id
'''
items_products_df = pd.read_sql_query(query_items_products, conn)
items_products_df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,e5f2d52b802189ee658865ca93d83a8f,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,c777355d18b72b67abbeef9df44fd0fd,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,7634da152a4610f1595efa32f14722fc,perfumaria,42.0,480.0,1.0,200.0,16.0,10.0,15.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,ac6c3623068f30de03045865e4e10089,ferramentas_jardim,59.0,409.0,1.0,3750.0,35.0,40.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,4aa6014eceb682077f9dc4bffebc05b0,utilidades_domesticas,43.0,1002.0,3.0,10150.0,89.0,15.0,40.0
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,32e07fd915822b0765e448c4dd74c828,informatica_acessorios,31.0,232.0,1.0,8950.0,45.0,26.0,38.0
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,72a30483855e2eafc67aee5dc2560482,esporte_lazer,43.0,869.0,1.0,967.0,21.0,24.0,19.0
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,9c422a519119dcad7575db5af1ba540e,informatica_acessorios,56.0,1306.0,1.0,100.0,20.0,20.0,20.0


### orders y order_payments
### [BBT3-19](https://alkemy-labs.atlassian.net/browse/BBT3-19) Judith

In [10]:
query_order_payments = '''
  SELECT * FROM orders
  INNER JOIN order_payments
  ON orders.order_id = order_payments.order_id
'''
query_order_payments = pd.read_sql_query(query_order_payments, conn)
query_order_payments

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,0a8556ac6be836b46b3e89920d59291c,delivered,2018-04-25 22:01:49,2018-04-25 22:15:09,2018-05-02 15:20:00,2018-05-09 17:36:51,2018-05-22 00:00:00,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,f2c7fc58a9de810828715166c672f10a,delivered,2018-06-26 11:01:38,2018-06-26 11:18:58,2018-06-28 14:18:00,2018-06-29 20:32:09,2018-07-16 00:00:00,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,25b14b69de0b6e184ae6fe2755e478f9,delivered,2017-12-12 11:19:55,2017-12-14 09:52:34,2017-12-15 20:13:22,2017-12-18 17:24:41,2018-01-04 00:00:00,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,7a5d8efaaa1081f800628c30d2b0728f,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,2017-12-21 01:35:51,2018-01-04 00:00:00,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,15fd6fb8f8312dbb4674e4518d6fa3b3,delivered,2018-05-21 13:59:17,2018-05-21 16:14:41,2018-05-22 11:46:00,2018-06-01 21:44:53,2018-06-13 00:00:00,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,5d576cb2dfa3bc05612c392a1ee9c654,delivered,2018-03-08 16:57:23,2018-03-10 03:55:25,2018-03-12 18:19:36,2018-03-16 13:09:51,2018-04-04 00:00:00,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,2079230c765a88530822a34a4cec2aa0,delivered,2017-08-18 09:45:35,2017-08-18 10:04:56,2017-08-18 18:04:24,2017-08-23 22:25:56,2017-09-12 00:00:00,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,e4abb5057ec8cfda9759c0dc415a8188,invoiced,2017-11-18 17:27:14,2017-11-18 17:46:05,None,None,2017-12-05 00:00:00,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,5d719b0d300663188169c6560e243f27,delivered,2018-08-07 23:26:13,2018-08-07 23:45:00,2018-08-09 11:46:00,2018-08-21 22:41:46,2018-08-24 00:00:00,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


### Extracción de datos desde las tablas customers y orders
### [BBT3-15](https://alkemy-labs.atlassian.net/browse/BBT3-15) - Fabricio

In [11]:
query_customers_orders = '''
SELECT *
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
'''
customers_and_orders_df = pd.read_sql_query(query_customers_orders, conn)

In [12]:
customers_and_orders_df.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,b0830fb4747a6c6d20dea0b8c802d7ef,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,41ce2a54c0b03bf3443c3d931a367089,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,f88197465ea7920adcdbec7375364d82,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,8ab97904e6daea8866dbdbc4fb7aad2c,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


### Extracción de datos desde las tablas orders y orders_items
### [BBT3-16](https://alkemy-labs.atlassian.net/browse/BBT3-16) - Fabricio

In [13]:
query_orders_and_order_items = '''
SELECT *
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
'''
orders_and_order_items_df = pd.read_sql_query(query_orders_and_order_items, conn)

In [14]:
orders_and_order_items_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


### Extracción de datos desde las tablas orders, payments y customers
### [BBT3-24](https://alkemy-labs.atlassian.net/browse/BBT3-24) - Fabricio

In [15]:
query_orders_payments_and_customers = '''
SELECT *
FROM orders o
JOIN order_payments op ON o.order_id = op.order_id
JOIN customers c ON o.customer_id = c.customer_id
'''
orders_payments_and_customers_df = pd.read_sql_query(query_orders_payments_and_customers, conn)

In [16]:
orders_payments_and_customers_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_id,payment_sequential,payment_type,payment_installments,payment_value,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,b81ef226f3fe1789b1e8b2acac839d17,0a8556ac6be836b46b3e89920d59291c,delivered,2018-04-25 22:01:49,2018-04-25 22:15:09,2018-05-02 15:20:00,2018-05-09 17:36:51,2018-05-22 00:00:00,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33,0a8556ac6be836b46b3e89920d59291c,708ab75d2a007f0564aedd11139c7708,39801,teofilo otoni,MG
1,a9810da82917af2d9aefd1278f1dcfa0,f2c7fc58a9de810828715166c672f10a,delivered,2018-06-26 11:01:38,2018-06-26 11:18:58,2018-06-28 14:18:00,2018-06-29 20:32:09,2018-07-16 00:00:00,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39,f2c7fc58a9de810828715166c672f10a,a8b9d3a27068454b1c98cc67d4e31e6f,2422,sao paulo,SP
2,25e8ea4e93396b6fa0d3dd708e76c1bd,25b14b69de0b6e184ae6fe2755e478f9,delivered,2017-12-12 11:19:55,2017-12-14 09:52:34,2017-12-15 20:13:22,2017-12-18 17:24:41,2018-01-04 00:00:00,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71,25b14b69de0b6e184ae6fe2755e478f9,6f70c0b2f7552832ba46eb57b1c5651e,2652,sao paulo,SP
3,ba78997921bbcdc1373bb41e913ab953,7a5d8efaaa1081f800628c30d2b0728f,delivered,2017-12-06 12:04:06,2017-12-06 12:13:20,2017-12-07 20:28:28,2017-12-21 01:35:51,2018-01-04 00:00:00,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78,7a5d8efaaa1081f800628c30d2b0728f,87695ed086ebd36f20404c82d20fca87,36060,juiz de fora,MG
4,42fdf880ba16b47b59251dd489d4441a,15fd6fb8f8312dbb4674e4518d6fa3b3,delivered,2018-05-21 13:59:17,2018-05-21 16:14:41,2018-05-22 11:46:00,2018-06-01 21:44:53,2018-06-13 00:00:00,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45,15fd6fb8f8312dbb4674e4518d6fa3b3,4291db0da71914754618cd789aebcd56,18570,conchas,SP


### Extracción de datos desde las tablas orders y reviews

### [BBT3-22](https://alkemy-labs.atlassian.net/browse/BBT3-22) Pablo


Crear un DataFrame que contenga el JOIN de la tabla orders y tabla reviews.

In [17]:
query_orders_reviews = '''
SELECT *
FROM orders AS o
INNER JOIN order_reviews AS orv
  ON o.order_id = orv.order_id
;
'''
orders_reviews_df = pd.read_sql_query(query_orders_reviews, conn)
orders_reviews_df.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,73fc7af87114b39712e6da79b0a377eb,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
1,a548910a1c6147796b98fdf73dbeba33,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
2,f9e4b658b201a9f2ecdecbb34bed034b,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24


### Extracción de datos desde las tablas orders_items, products y products_category_translation

### [BBT3-23](https://alkemy-labs.atlassian.net/browse/BBT3-23) Pablo


Crear un DataFrame que contenga el JOIN de la tabla orders_items, tabla products y product_category_name_translation.  

In [18]:
query_ordersitems_products = '''
SELECT *
FROM order_items AS oi
INNER JOIN products AS p
  ON oi.product_id = p.product_id
INNER JOIN product_category_name_translation AS pct
  ON p.product_category_name = pct.product_category_name
GROUP BY p.product_category_name
;
'''
orderitems_products_df = pd.read_sql_query(query_ordersitems_products, conn)
orderitems_products_df.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name,product_category_name_english
0,00137e170939bba5a3134e2386413108,1,672e757f331900b9deea127a2a7b79fd,e59aa562b9f8076dd550fcddf0e73491,2017-11-30 06:30:55,397.0,24.65,672e757f331900b9deea127a2a7b79fd,agro_industria_e_comercio,39.0,383.0,1.0,2650.0,30.0,30.0,20.0,agro_industria_e_comercio,agro_industry_and_commerce
1,00130c0eee84a3d909e75bc08c5c3ca1,1,89321f94e35fc6d7903d36f74e351d40,16090f2ca825584b5a147ab24aa30c86,2018-06-14 05:16:24,27.9,7.94,89321f94e35fc6d7903d36f74e351d40,alimentos,59.0,982.0,1.0,150.0,17.0,13.0,13.0,alimentos,food
2,01ae5e4c5d2549f7c05000a2f5856c28,1,84f5c4f480ad6c9998d6a6860f1a2e41,282f23a9769b2690c5dda22e316f9941,2017-11-09 15:15:23,39.0,16.60,84f5c4f480ad6c9998d6a6860f1a2e41,alimentos_bebidas,56.0,963.0,1.0,1100.0,16.0,15.0,15.0,alimentos_bebidas,food_drink


### Extracción de datos desde las tablas customers y geolocations

### [BBT3-20](https://alkemy-labs.atlassian.net/browse/BBT3-20) Pablo

Crear un DataFrame que contenga el JOIN de la tabla customers y tabla geolocation.

In [25]:
# Hacemos el promedio de coordenadas por cada zip_code y calculamos tambien la desviacion standard de esas
# coordenadas para evitar conflictos en futuras tareas
# Desviacion standard calculada como: std = (avg(col*col) - avg(col)*avg(col))^(1/2)
# siendo std^2 la varianza

query_geolocation_customers = '''
SELECT *
FROM (
  SELECT geolocation_zip_code_prefix, AVG(geolocation_lat) AS mean_lat, AVG(geolocation_lng) as mean_lng,
          SQRT(avg(geolocation_lat * geolocation_lat) - avg(geolocation_lat)*avg(geolocation_lat)) AS std_lat,
          SQRT(avg(geolocation_lng * geolocation_lng) - avg(geolocation_lng)*avg(geolocation_lng)) AS std_lng
  FROM geolocation
  GROUP BY geolocation_zip_code_prefix
  ) AS geoavg
INNER JOIN customers AS c
  ON c.customer_zip_code_prefix = geoavg.geolocation_zip_code_prefix
;
'''

geolocation_customer_df = pd.read_sql_query(query_geolocation_customers, conn)
geolocation_customer_df.head(3)

,geolocation_zip_code_prefix,mean_lat,mean_lng,std_lat,std_lng,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,14409,-20.498489,-47.396929,0.013494,0.005581,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,9790,-23.727992,-46.542848,0.007551,0.009140,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,1151,-23.531642,-46.656289,0.001380,0.001577,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP


### Extracción de datos desde las tablas orders y geolocations

### [BBT3-21](https://alkemy-labs.atlassian.net/browse/BBT3-21) Pablo

Crear un DataFrame que contenga el JOIN de la tabla orders y tabla geolocation.  

In [26]:
# Hacemos el promedio de coordenadas por cada zip_code y calculamos tambien la desviacion standard de esas
# coordenadas para evitar conflictos en futuras tareas
# Desviacion standard calculada como: std = (avg(col*col) - avg(col)*avg(col))^(1/2)
# siendo std^2 la varianza

query_geolocation_orders = '''
SELECT *
FROM (
  SELECT geolocation_zip_code_prefix, AVG(geolocation_lat) AS mean_lat, AVG(geolocation_lng) as mean_lng,
          SQRT(avg(geolocation_lat * geolocation_lat) - avg(geolocation_lat)*avg(geolocation_lat)) AS std_lat,
          SQRT(avg(geolocation_lng * geolocation_lng) - avg(geolocation_lng)*avg(geolocation_lng)) AS std_lng
  FROM geolocation
  GROUP BY geolocation_zip_code_prefix
  ) AS geoavg
INNER JOIN customers AS c
  ON c.customer_zip_code_prefix = geoavg.geolocation_zip_code_prefix
INNER JOIN orders AS o
  ON c.customer_id = o.customer_id
;
'''

geolocation_orders_df = pd.read_sql_query(query_geolocation_orders, conn)
geolocation_orders_df.head(3)

,geolocation_zip_code_prefix,mean_lat,mean_lng,std_lat,std_lng,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,3149,-23.576983,-46.587161,0.002764,0.000383,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,47813,-12.177924,-44.660711,0.181920,1.462450,b0830fb4747a6c6d20dea0b8c802d7ef,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,75265,-16.745150,-48.514783,0.003059,0.005126,41ce2a54c0b03bf3443c3d931a367089,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
